# Programma che effettua lo scraping agli articoli di datajournalism sul sito Dataroom del Corriere della Sera

In [14]:

import time, csv               # sono importate le diverse librerie per il corretto funzionamento del bot
import pandas as pd
from selenium import webdriver
from datetime import datetime


driver = webdriver.Chrome()    # il driver è associato a Chrome

driver.get("https://www.corriere.it/dataroom-milena-gabanelli/") # home page di Dataroom, rubrica del Corriere curato da Milena Gabanelli che tratta di articoli di datajournalism 

    
articoli = driver.find_elements_by_class_name("md_ap_sp2") # classe degli articoli presenti nella pagina


lista_articoli = []   # un array che conterrà i valori del dizionario art_item1
solo_Link = []        # un array dove saranno inseriti esclusivamente i link degli articoli

for articolo in articoli:    # scorre gli articoli per ottenere i rispettivi link
    
    link = articolo.find_element_by_xpath('.//a').get_attribute('href')  # . (seleziona il nodo corrente) poichè senza si ripete il primo
    
    
    art_item1 = {            # primo dizionario che contiene per ogni articolo il Link
        
        'Link': link,
        
    }
    lista_articoli.append(art_item1)  # il primo dizionario è appeso a lista_articoli
    solo_Link.append(link)            # ogni link è appeso all'array solo_Link

driver.quit()          # il bot si arresta
df1 = pd.DataFrame(lista_articoli) # tramite la libreria Pandas, la funzione DataFrame() trasforma  lista_articoli in una tabella di oggetti eterogenei


corpi = []           # array che conterrà i "corpi degli articoli"

for indirizzo in solo_Link:       # scorre ogni link della lista solo_Link
    driver = webdriver.Chrome() 

    driver.get(indirizzo)     # il bot accede all'indirizzo
                              

    titolo = driver.find_element_by_xpath('.//*[@class="article-title"]').text
    author = driver.find_elements_by_xpath('.//*[@class="show-user"]')
    lista_autori = []     
    
    for elemento in author:                 # scorre gli elementi della lista author              
        lista_autori.append(elemento.text)  # prende il testo dell'elemento appendendolo all'array lista_autori
    
    autori = ' '.join(map(str, lista_autori)) # map() prende in ingresso una o più liste e una funzione. Per ogni elemento presente in ciascuna lista specificata, viene applicata la funzione str() che li converte in stringhe.
        
    
    
    now = datetime.now() # data nel formato Americano e orario corrente  
    timestamp = now.strftime("%d/%m/%Y, %H:%M:%S") # Il metodo strftime () restituisce una stringa che rappresenta la data e l'ora utilizzando l'oggetto date, time o datetime
    
     
    data_pubblicazione = driver.find_elements_by_xpath('.//*[@class="article-date-place"]')    
    lista_data = []
    
    for elemento in data_pubblicazione:        # scorre gli elementi della lista data_pubblicazione
        lista_data.append(elemento.text)       # prende il testo dell'elemento appendendolo all'array lista_data
    
    data = ' '.join(map(str, lista_data))      # map() prende in ingresso una o più liste e una funzione. Per ogni elemento presente in ciascuna lista specificata, viene applicata la funzione str() che lo converte in stringa.


    elementip = driver.find_elements_by_tag_name("p") # lista intera di tutti gli elementi p

    paragrafi = []                       # andrà a contenere l'insieme dei paragrafi che comporranno un singolo articolo
    
    for elemento in elementip:           # scorre gli elementi della lista elementip
        paragrafi.append(elemento.text)  # prende il testo dell'elemento appendendolo all'array paragrafi
    #print(paragrafi)

    testo = ' '.join(map(str, paragrafi))   # map() prende in ingresso una o più liste e una funzione. Per ogni elemento presente in ciascuna lista specificata, viene applicata la funzione str() che lo converte in stringa.



    art_item2 = {                        # secondo dizionario che contiene per ogni articolo: titolo, autori, data, timestamp dello scraping e testo
            
            'Titolo': titolo,        
            'Autori': autori,
            'Data': data,
            'Timestamp dello scraping': timestamp,
            'Testo': testo        
        
       }
    
    corpi.append(art_item2)      # il secondo dizionario è appeso all' array corpi

    driver.quit()         # il bot si arresta
df2 = pd.DataFrame(corpi) # tramite la libreria Pandas, la funzione DataFrame() trasforma la lista corpi in una tabella di oggetti eterogenei

dfTotale = pd.concat([df1,df2], axis = 1) # i due dataframe sono concatenati in un unico dataframe
dfTotale.to_csv(r'C:\Users\robeo\Desktop\dataframeDataRoomcompleto.csv', index = True, header = True) # il dataframe complessivo è convertito nel formato CSV
